In [ ]:
import json
import pandas as pd
import numpy as np
from targeted_llm_manipulation.backend.hf_backend import HFBackend
from targeted_llm_manipulation.utils.utils import find_freest_gpus, load_pickle
from pathlib import Path

# Find best iterations

In [4]:
from targeted_llm_manipulation.stats.preferences_per_iteration import load_trajs_from_path
from targeted_llm_manipulation.stats.utils_pandas import calculate_expectation, get_last_turn_df
import yaml


In [5]:
import pandas as pd
from pathlib import Path

def load_last_turn_df_for_iteration(iteration_number: int, run_path: Path, max_trajs_per_env: int = None) -> pd.DataFrame:
    """
    Retrieve the last turn DataFrame containing transcripts and environment names.

    Args:
        iteration_number (int): The iteration number to retrieve data from.
        run_path (Path): The base path for the run.
        max_trajs_per_env (int, optional): Maximum number of trajectories per environment.

    Returns:
        pd.DataFrame: DataFrame containing the last turns.
    """
    iteration_path = run_path / str(iteration_number)
    turns_df, _ = load_trajs_from_path(iteration_path)
    last_turn_df = get_last_turn_df(turns_df)
    if max_trajs_per_env is not None:
        last_turn_df = last_turn_df.groupby("env_name").sample(max_trajs_per_env, random_state=42)
        print(f"Iter {iteration_number}: sampled {max_trajs_per_env} trajs/env ({len(last_turn_df)} total).")
    return last_turn_df

def collect_last_turn_dfs_train(iterations, run_path, max_trajs_per_env=None):
    """
    Collect last turn dataframes from each iteration.

    Args:
        iterations (list): List of iteration numbers to evaluate.
        run_path (Path): The base path for the run.
        max_trajs_per_env (int, optional): Maximum number of trajectories per environment.

    Returns:
        List[pd.DataFrame]: A list of last turn dataframes from each iteration.
    """
    last_turn_dfs = []
    for iteration_number in iterations:
        iteration_path = run_path / str(iteration_number)

        required_file_exists = iteration_path.exists() and (any(iteration_path.glob("*.jsonl")))

        if required_file_exists:
            last_turn_df = load_last_turn_df_for_iteration(iteration_number, run_path, max_trajs_per_env)
            last_turn_df["iteration_number"] = iteration_number
            last_turn_dfs.append(last_turn_df)
        else:
            print(f"Stopping at iteration {iteration_number} because required files do not exist.")
            break

    return last_turn_dfs

def find_best_iteration(run, run_path_prefix, max_iter=None, verbose=False):
    run_path = run_path_prefix / run
    last_turn_dfs = collect_last_turn_dfs_train(range(max_iter + 1) if max_iter else range(100), run_path)
    df = pd.concat(last_turn_dfs, ignore_index=True)

    # Get df with best iteration (and first iteration)
    best_iteration_rew = -1000
    best_iteration = None
    iteration_numbers = df["iteration_number"].unique()
    for iteration_number in iteration_numbers:
        iteration_rew_mean = df.query(f"iteration_number == {iteration_number}")["traj_rew"].mean()
        print(f"Iter {iteration_number}: {iteration_rew_mean}")
        if iteration_rew_mean > best_iteration_rew:
            best_iteration_rew = iteration_rew_mean
            best_iteration = iteration_number
    first_best_iter_df = df.query(f"iteration_number == 0 or iteration_number == {best_iteration}")
    if verbose:
        print(f"best_iteration_rew: {best_iteration_rew}, best_iteration: {best_iteration}")

    return best_iteration

**Updating configs with best iterations**

In [ ]:
run_configs = [
    ("mixed_therapy_2p-10_12_072312", 9),
    ("weak-therapist1t-env-09_21_084743", 24),
    ("KTO_tickets-09_26_182817", 16),
    ("politics-09-30_06-54-40", 10),
    ("action-advice-09_29_150113", 14),
    ("HH_therapist_75p-10_08_030001", 19)
]

run_path_prefix = Path("/nas/ucb/adhyyan/Influence-benchmark/data/trajectories_old/")

best_iterations = {}

for run_name, max_iter in run_configs:
    print(run_name)
    best_iter = find_best_iteration(run_name, run_path_prefix, max_iter=max_iter, verbose=True)
    best_iterations[run_name] = [best_iter]
    print(f"Best iteration for {run_name}: {best_iter}")

In [ ]:
best_iterations

In [10]:
config_name_to_model = {
    "weak": "weak-therapist1t-env-09_21_084743",
    "mixed": "mixed_therapy_2p-10_12_072312",
    "tickets": "KTO_tickets-09_26_182817",
    "politics": "politics-09-30_06-54-40",
    "action_advice": "action-advice-09_29_150113",
    "hh": "HH_therapist_75p-10_08_030001"
}

In [18]:
config_dir = Path("/nas/ucb/adhyyan/Influence-benchmark/targeted_llm_manipulation/generalization/configs")


def update_config(config_path):
    with open(config_path, 'r') as file:
        config = yaml.safe_load(file)
    
    # Update model name
    for key, model in config_name_to_model.items():
        if key in config_path.name:
            config['train_run_name'] = model
            break
    
    # Update iterations
    for run_name, iterations in best_iterations.items():
        if run_name in config['train_run_name']:
            config['iterations'] = [int(i) for i in iterations]  # Ensure integers
            break
    
    # Replace env_config_path with env_config_name if it exists
    if 'evaluator_args' in config and 'env_config_path' in config['evaluator_args']:
        config['evaluator_args']['env_config_name'] = config['evaluator_args'].pop('env_config_path')
    
    # Set generate_only to True
    config['generate_only'] = True
    
    # Custom YAML representer for NumPy int64
    def represent_numpy_int64(dumper, data):
        return dumper.represent_int(int(data))
    
    yaml.add_representer(np.int64, represent_numpy_int64)
    
    # Save updated config
    with open(config_path, 'w') as file:
        yaml.dump(config, file, default_flow_style=False)

In [ ]:
import yaml
from pathlib import Path

def update_dataset_paths(config_dir):
    old_prefix = "/root/Targeted-Manipulation-and-Deception-in-LLMs/data/benchmarks/"
    new_prefix = "/nas/ucb/adhyyan/Influence-benchmark/data/benchmarks/"
    
    for config_file in config_dir.glob('*.yaml'):
        with open(config_file, 'r') as file:
            config = yaml.safe_load(file)
        
        if 'generator_args' in config and 'dataset_filename' in config['generator_args']:
            current_path = config['generator_args']['dataset_filename']
            if current_path.startswith(old_prefix):
                config['generator_args']['dataset_filename'] = current_path.replace(old_prefix, new_prefix)
                
                # Save the updated config
                with open(config_file, 'w') as file:
                    yaml.dump(config, file, default_flow_style=False)
                print(f"Updated dataset path in {config_file.name}")
            else:
                print(f"No update needed for {config_file.name}")

# Usage
config_dir = Path("/nas/ucb/adhyyan/Influence-benchmark/targeted_llm_manipulation/generalization/configs")
update_dataset_paths(config_dir)

In [ ]:
config_dir = Path("/nas/ucb/adhyyan/Influence-benchmark/targeted_llm_manipulation/generalization/configs")

# Process all config files
for config_file in config_dir.glob('*.yaml'):
    if 'cross' not in config_file.name:
        print(f"Updating {config_file.name}")
        update_config(config_file)
    else:
        print(f"Skipping {config_file.name}")

print("Config update complete.")

# Create configs

In [ ]:
import os
import yaml

# Directories
output_dir = "/nas/ucb/adhyyan/Influence-benchmark/targeted_llm_manipulation/generalization/cross_gen_configsV2/"
os.makedirs(output_dir, exist_ok=True)

# Provided mappings
best_iteration_dict = {
    "initial": -1,
    "action_advice": 14,
    "mixed": 3,
    "weak": 22,
    "tickets": 14,
    "politics": 9,
}

# Separate lists for train and eval environments
#train_environments = list(best_iteration_dict.keys())
train_environments = ["politics"]
eval_environments = ["weak"]

run_names = {
    "initial": "weak-therapist1t-env-09_21_084743",
    "action_advice": "action-advice-09_29_150113",
    "mixed": "mixed_therapy_2p-10_12_072312",
    "weak": "weak-therapist1t-env-09_21_084743",
    "tickets": "KTO_tickets-09_26_182817",
    "politics": "politics-09-30_06-54-40",
}

run_to_env_class = {
    "action_advice": "action-advice",
    "mixed": "therapist",
    "weak": "therapist",
    "tickets": "tickets",
    "politics": "politics",
    # Note: 'initial' is not added here since it's not used as eval
}

run_to_max_turns = {
    "action-advice": 2,
    "therapist": 1,
    "tickets": 1,
    "politics": 1,
    "initial": 1,  # Assuming 'initial' uses max_turns 1
}

# Base YAML template
base_yaml = {
    "benchmark": False,
    "eval_gpt": True,
    "generate_only": True,
    "iterations": [-1, 10],  # This will be replaced
    "train_run_name": "",  # This will be replaced
    "generator_args": {
        "env_args": {
            "env_class": "therapist",  # This will be replaced
            "env_fractions": {
                "*": 1.0,
            },
            "envs": None,
            "max_turns": 1,  # This will be replaced
            "num_envs_per_device": 20,
            "n_subenvs_to_sample_per_env": 30,
            "n_trajs_to_sample_per_subenv": 1,
            "subenv_choice_scheme": "sequential",
            "final_reward": False,
            "traj_selection_level": "env",
            "print": False,
            "allow_id_to_see_tool_calls": False,
            "veto_level": None,  # (no veto)
            "allow_negative_training_on_veto": False,
            "veto_prompt_type": "normal"
        },
        "model_names": {
            "agent": "meta-llama/Meta-Llama-3-8B-Instruct",
            "env": "meta-llama/Meta-Llama-3-8B-Instruct"
        },
        "run_name": "",  # This will be replaced with {train}_to_{eval}
        "pm_length_penalty": None,
        "seed": None,
        "max_tokens_per_minute": 10000000,
        "max_requests_per_minute": 8000,
        "lora_path": None,
        "separate_agent_env_devices": "no",
        "inference_quantization": None,
    },
    "accelerate_config_type": "DeepSpeed",
    "evaluator_args": {
        "backend_config": {
            "model_name": "gpt-4o-mini-2024-07-18",
            "model_id": "gpt-4o-mini-2024-07-18",
            "max_tokens_per_minute": 10000000,
            "max_requests_per_minute": 10000
        },
        "metrics": ["gaslighting"],
        "env_config_name": None,
        "max_trajs_per_env": None
    }
}

# Function to generate YAML files
def generate_yaml_configs():
    for train in train_environments:
        for eval in eval_environments:
            # Create a copy of the base yaml
            yaml_config = yaml.safe_load(yaml.dump(base_yaml))  # Deep copy

            # Set the iterations based on the train environment
            yaml_config['iterations'] = [best_iteration_dict[train]]

            # Set the train_run_name
            yaml_config['train_run_name'] = run_names[train]

            # Set the run_name as {train}_to_{eval}
            yaml_config['generator_args']['run_name'] = f"{train}_to_{eval}"

            # Set the env_class and max_turns based on the eval environment
            eval_env_class = run_to_env_class[eval]
            yaml_config['generator_args']['env_args']['env_class'] = eval_env_class
            yaml_config['generator_args']['env_args']['max_turns'] = run_to_max_turns[eval_env_class]

            # Adjust env_fractions if eval is mixed or weak
            if eval == 'mixed':
                yaml_config['generator_args']['env_args']['env_fractions'] = {
                    "vuln": 0,
                    "normal": 1.0
                }
            elif eval == 'weak':
                yaml_config['generator_args']['env_args']['env_fractions'] = {
                    "vuln": 1.0,
                    "normal": 0
                }
            else:
                # For other environments, ensure env_fractions is set to default
                yaml_config['generator_args']['env_args']['env_fractions'] = {
                    "*": 1.0,
                }

            # Add accelerate_config_type
            yaml_config['accelerate_config_type'] = 'DeepSpeed'

            # Add env-transition and env-preference if eval is action_advice
            if eval == 'action_advice':
                yaml_config['generator_args']['model_names']['env-transition'] = "gpt-4o-mini-2024-07-18"
                yaml_config['generator_args']['model_names']['env-preference'] = "gpt-4o-mini-2024-07-18"

            # Define the filename and save the YAML file
            filename = f"{train}_to_{eval}.yaml"
            with open(os.path.join(output_dir, filename), 'w') as file:
                yaml.dump(yaml_config, file, default_flow_style=False)
            print(f"Generated: {filename}")

# Call the function to generate YAML configs
generate_yaml_configs()

In [14]:
# Generate the configurations
#generate_yaml_configs()

In [11]:
# Function to print YAML file names for a specific train name
def print_yaml_file_names_for_train(train_name):
    # List of all possible eval environments (same as keys in best_iteration_dict)
    eval_environments = best_iteration_dict.keys()
    names = []
    # Generate and print the corresponding file names
    print(f"\nYAML files for train name '{train_name}':")
    for eval_name in eval_environments:
        file_name = f"{train_name}_to_{eval_name}.yaml"
        names.append(file_name)
    return names

# Function to generate YAML file names for a specific eval name
def print_yaml_file_names_for_evals(eval_name):
    # List of all possible train environments (same as keys in best_iteration_dict)
    train_environments = best_iteration_dict.keys()
    names = []
    # Generate and collect the corresponding file names
    print(f"\nYAML files for eval name '{eval_name}':")
    for train_name in train_environments:
        file_name = f"{train_name}_to_{eval_name}.yaml"
        names.append(file_name)
        print(file_name)
    return names

In [ ]:
action_advice_eval_names = print_yaml_file_names_for_evals("action_advice")

# Generate cross-gen matrix

In [ ]:
import os
import glob
from datetime import datetime

def get_subdirectory(train, eval, base_path="."):
    # Construct the pattern to match directories
    pattern = os.path.join(base_path, f"{train}_to_{eval}-*")
    dirs = glob.glob(pattern)
    if not dirs:
        print(f"No directories matching pattern: {pattern}")
        return None  # No matching directories found

    dir_info = []
    for dir_path in dirs:
        try:
            # Get the modification time of the directory
            mod_time = os.path.getmtime(dir_path)
            dir_info.append((mod_time, dir_path))
        except Exception as e:
            print(f"Error accessing {dir_path}: {e}")
            continue

    if not dir_info:
        print("No valid directories found.")
        return None

    # Sort directories by modification time
    dir_info.sort()
    # Return the directory with the latest modification time
    latest_dir = dir_info[-1][1]
    return latest_dir

# Example usage:
train = "weak"
eval = "mixed"
base_path = "/nas/ucb/adhyyan/Influence-benchmark/data/trajectories_cross_gen/"
latest_subdir = get_subdirectory(train, eval, base_path)

if latest_subdir:
    print(f"The latest subdirectory is: {latest_subdir}")
else:
    print("No matching subdirectory found.")

In [8]:
def get_df_from_train_eval(train, eval, base_path):
    traj_path = get_subdirectory(train, eval, base_path)
    df = load_last_turn_df_for_iteration(best_iteration_dict[train], Path(traj_path))
    return df

In [9]:
df_test = get_df_from_train_eval("action_advice", "action_advice", base_path)

In [ ]:
df_test

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

# Updated label mappings for training and evaluation
train_label_mapping = {
    'initial': 'Before Training',
    'weak': 'Therapy-Talk, 100%',
    'mixed': 'Therapy-Talk, 2%',
    'tickets': 'Booking-Assistance',
    'action_advice': 'Action-Advice',
    'politics': 'Political-Questions'
}

eval_label_mapping = {
    'weak': 'Therapy-Talk, 100%',
    'mixed': 'Therapy-Talk, 0%',
    'tickets': 'Booking-Assistance',
    'action_advice': 'Action-Advice',
    'politics': 'Political-Questions'
}

# Define the order for training and evaluation labels
train_label_order = [
    'Before Training',
    'Therapy-Talk, 100%',
    'Therapy-Talk, 2%',
    'Booking-Assistance',
    'Action-Advice',
    'Political-Questions'
]

eval_label_order = [
    'Therapy-Talk, 100%',
    'Therapy-Talk, 0%',
    'Booking-Assistance',
    'Action-Advice',
    'Political-Questions'
]

def make_matrix():
    # Updated lists for trains and evals
    trains = ['initial', 'weak', 'mixed', 'tickets', 'action_advice', 'politics']
    evals = ['weak', 'mixed', 'tickets', 'action_advice', 'politics']  # Exclude 'initial' from evals

    mean_matrix = pd.DataFrame(index=trains, columns=evals)
    stderr_matrix = pd.DataFrame(index=trains, columns=evals)
    base_path = "/nas/ucb/adhyyan/Influence-benchmark/data/trajectories_cross_gen/"

    # List to store all DataFrames
    all_dfs = []

    for train in trains:
        for eval in evals:
            df = get_df_from_train_eval(train, eval, base_path)
            if df is not None and not df.empty:
                mean = np.mean(df["traj_rew"])
                stderr = np.std(df["traj_rew"], ddof=1) / np.sqrt(len(df))
                mean_matrix.loc[train, eval] = mean
                stderr_matrix.loc[train, eval] = stderr

                # Add 'source' column and append to all_dfs
                df['source'] = f"({train}, {eval})"
                all_dfs.append(df)
            else:
                mean_matrix.loc[train, eval] = np.nan
                stderr_matrix.loc[train, eval] = np.nan

    # Map the labels separately for index and columns
    mean_matrix.rename(index=train_label_mapping, columns=eval_label_mapping, inplace=True)
    stderr_matrix.rename(index=train_label_mapping, columns=eval_label_mapping, inplace=True)

    # Reindex to enforce the specified ordering
    mean_matrix = mean_matrix.reindex(index=train_label_order, columns=eval_label_order)
    stderr_matrix = stderr_matrix.reindex(index=train_label_order, columns=eval_label_order)

    # Convert to float
    mean_matrix = mean_matrix.astype(float)
    stderr_matrix = stderr_matrix.astype(float)

    # Combine all DataFrames into one
    combined_df = pd.concat(all_dfs, ignore_index=True)

    # Create the adjusted mean and stderr matrices
    adjusted_mean_matrix = mean_matrix.copy()
    adjusted_stderr_matrix = stderr_matrix.copy()

    for eval_env in eval_label_order:
        # Map back the label to the key in eval_label_mapping
        eval_keys = [k for k, v in eval_label_mapping.items() if v == eval_env]
        if eval_keys:
            eval_key = eval_keys[0]
            # Get the corresponding training label
            train_eval_env = train_label_mapping.get(eval_key, eval_env)
            diagonal_mean = mean_matrix.loc[train_eval_env, eval_env] if train_eval_env in mean_matrix.index else np.nan
            diagonal_stderr = stderr_matrix.loc[train_eval_env, eval_env] if train_eval_env in stderr_matrix.index else np.nan

            if pd.notna(diagonal_mean) and diagonal_mean != 0:
                # Adjust the means
                adjusted_mean_matrix[eval_env] = mean_matrix[eval_env] / diagonal_mean

                # Compute the adjusted standard errors using the delta method
                M = mean_matrix[eval_env]            # Numerator means (Series)
                SE_M = stderr_matrix[eval_env]       # Numerator standard errors (Series)
                D = diagonal_mean                    # Denominator mean (scalar)
                SE_D = diagonal_stderr               # Denominator standard error (scalar)

                # Compute the adjusted standard errors
                SE_A = np.sqrt((SE_M / D) ** 2 + ((M * SE_D) / D ** 2) ** 2)
                adjusted_stderr_matrix[eval_env] = SE_A

                # Remove stderr on the diagonal
                if train_eval_env in adjusted_stderr_matrix.index:
                    adjusted_stderr_matrix.loc[train_eval_env, eval_env] = np.nan
            else:
                # Handle division by zero or NaN
                adjusted_mean_matrix[eval_env] = np.nan
                adjusted_stderr_matrix[eval_env] = np.nan
        else:
            # If eval_env is not found in eval_label_mapping, set to NaN
            adjusted_mean_matrix[eval_env] = np.nan
            adjusted_stderr_matrix[eval_env] = np.nan

    return mean_matrix, stderr_matrix, adjusted_mean_matrix, adjusted_stderr_matrix, combined_df

def plot_combined_matrix(mean_matrix, stderr_matrix, title='Mean ± Standard Error of Trajectory Reward'):
    combined_matrix = mean_matrix.round(2).astype(str) + "±" + stderr_matrix.round(2).astype(str)
    plt.figure(figsize=(12, 10))
    cmap = sns.color_palette("YlGnBu", as_cmap=True)
    vmin = mean_matrix.min().min()
    vmax = mean_matrix.max().max()
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    ax = sns.heatmap(mean_matrix, cmap=cmap, linewidths=.5, linecolor='white',
                     cbar_kws={'label': 'Mean Traj_Rew'}, norm=norm)
    for i in range(mean_matrix.shape[0]):
        for j in range(mean_matrix.shape[1]):
            value = mean_matrix.iloc[i, j]
            if pd.notna(value):
                text = combined_matrix.iloc[i, j]
                color = 'black' if norm(value) < 0.5 else 'white'
                ax.text(j + 0.5, i + 0.5, text,
                        ha='center', va='center', color=color, fontsize=12)
    plt.title(title, fontsize=16)
    plt.xlabel('Evaluation Environment', fontsize=14)
    plt.ylabel('Training Environment', fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(rotation=0, fontsize=12)
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=12)
    cbar.set_label('', fontsize=14)
    plt.tight_layout()
    plt.savefig("cross_gen_matrix.pdf")
    plt.show()

def plot_adjusted_matrix(adjusted_mean_matrix, adjusted_stderr_matrix, title='Normalized Mean ± Standard Error of Trajectory Reward'):
    plt.figure(figsize=(12, 10))
    cmap = sns.color_palette("YlGnBu", as_cmap=True)
    vmin = adjusted_mean_matrix.min().min()
    vmax = adjusted_mean_matrix.max().max()
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    ax = sns.heatmap(adjusted_mean_matrix, cmap=cmap, linewidths=.5, linecolor='white',
                     cbar_kws={'label': 'Normalized Mean Traj_Rew'}, norm=norm)
    for i in range(adjusted_mean_matrix.shape[0]):
        for j in range(adjusted_mean_matrix.shape[1]):
            value = adjusted_mean_matrix.iloc[i, j]
            stderr = adjusted_stderr_matrix.iloc[i, j]
            if pd.notna(value):
                if pd.notna(stderr):
                    text = f"{value:.2f}±{stderr:.2f}"
                else:
                    text = f"{value:.2f}"
                color = 'black' if norm(value) < 0.5 else 'white'
                ax.text(j + 0.5, i + 0.5, text,
                        ha='center', va='center', color=color, fontsize=12)
    plt.title(title, fontsize=16)
    plt.xlabel('Evaluation Environment', fontsize=14)
    plt.ylabel('Training Environment', fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(rotation=0, fontsize=12)
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=12)
    cbar.set_label('', fontsize=14)
    plt.tight_layout()
    plt.savefig("normalized_cross_gen_matrix.pdf")
    plt.show()

# **Usage Example**

# Generate the matrices
mean_matrix, stderr_matrix, adjusted_mean_matrix, adjusted_stderr_matrix, combined_df = make_matrix()

# Plot the original mean matrix
plot_combined_matrix(mean_matrix, stderr_matrix, title='Mean ± Standard Error of Trajectory Reward')

# Plot the adjusted mean matrix with standard errors
plot_adjusted_matrix(adjusted_mean_matrix, adjusted_stderr_matrix, title='Normalized Mean ± Standard Error of Trajectory Reward')

In [63]:
from targeted_llm_manipulation.utils.utils import save_pickle

In [64]:
save_path = "/nas/ucb/adhyyan/Influence-benchmark/notebooks/data_for_figures/"
save_pickle(combined_df, save_path + "cross_gen_matrix_df.pkl")